**L'objectif ici est de comparer les notes données à la culture et la présence ou non de lieux culturels par arrondissement.**

In [ ]:
pip install openpyxl
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

Commençons par s'intéresser aux notes "culture" de chaque arrondissement :

In [ ]:
# Création d'un dataframe contenant les notations pour la culture de chaque arrondissement
Notes = pd.read_excel('/home/onyxia/Projet_Python_2AENSAE/Notes_arrondissements.xlsx', decimal=',', index_col=0)
NotesCulture = Notes['Culture']
NotesCulture

In [53]:
# Affichage des 4 meilleurs et des 4 pires arrondissement pour leur culture, d'après les notes des habitants
print('Les 4 arrondissements ayant les meilleures notes pour leur culture sont :')
print(NotesCulture.sort_values(ascending=False)[:4])

print('Les 4 arrondissements ayant les pires notes pour leur culture sont :')
print(NotesCulture.sort_values()[:4])

Les 4 arrondissements ayant les meilleures notes pour leur culture sont :
Paris 3e     9.33
Paris 1er    8.58
Paris 9e     8.50
Paris 5e     8.50
Name: Culture, dtype: float64
Les 4 arrondissements ayant les pires notes pour leur culture sont :
Paris 18e    5.89
Paris 10e    6.18
Paris 14e    6.54
Paris 20e    6.59
Name: Culture, dtype: float64


En ce qui concerne les lieux culturels, les Parisiens semblent donc avoir une préférence pour les arrondissements 3, 1, 9 et 5.
Le troisième arrondissement est largement en tête avec une note de 9.33.
Le premier, neuvième et cinquième arrondissement forment un deuxieme groupe en tête avec une note autour de 8.5.

Les arrondissements 18, 10, 14 et 20 obtiennent eux les pires notes : leurs notes sont réparties entre 5.89 pour le 18ème et 6.59.

Voyons quelques statistiques sur les notes données pour la culture :

In [ ]:
NotesCulture.describe()

Ces premières statistiques descriptives nous apprennent que le top 4 des arrondissement se situent au dessus de 75% des notes.
La note moyenne obtenue pour la culture est de 7.32, ce qui est haut. L'écart-type est de 0.93 ce qui est assez bas. Les notes sont
plutôt concentrées autour de 7. 

Si on compare donc les pires et meilleures notes, c'est surtout le 3ème arrondissement qui se démarque avec son 9.33. La pire note, 5.89, n'est pas autant éloignée du groupe que cela.

Maintenant, nous allons importer la base de données recensant tous les lieux culturels parisiens. Nous pourrons alors voir s'il y a une corrélation entre le nombre de lieux culturels et la satisfaction des habitants parisiens.

In [ ]:
# Import de la base de données de data.gouv.fr
LieuxCulturels = pd.read_csv('/home/onyxia/Projet_Python_2AENSAE/lieux_culturels.csv')
display(LieuxCulturels)

In [ ]:
# Suppression des '.0' dans la liste des codes postaux par soucis d'esthétisme :
LieuxCulturels.CODE_POSTAL = LieuxCulturels.CODE_POSTAL.astype(str)
LieuxCulturels.CODE_POSTAL = LieuxCulturels.CODE_POSTAL.str.replace('.0','')

In [50]:
# Suppression des lignes n'étant pas associées à un arrondissement :
LieuxCulturels.drop(LieuxCulturels[LieuxCulturels.CODE_POSTAL == 'nan'].index, inplace=True)

In [51]:
# Transformation de la base pour qu'elle recense uniquement le nombre de lieux culturels par arrondissement.
NbLieux = LieuxCulturels.CODE_POSTAL.value_counts()

In [52]:
# Affichage des 4 arrondissement ayant le plus (et le moins) de lieux culturels
print('Les 4 arrondissements ayant le plus de lieux culturels sont :')
print(NbLieux.sort_values(ascending=False)[:4])

print('Les 4 arrondissements ayant le moins de lieux culturels sont :')
print(NbLieux.sort_values()[:4])

Les 4 arrondissements ayant le plus de lieux culturels sont :
CODE_POSTAL
75011    90
75019    72
75018    71
75006    69
Name: count, dtype: int64
Les 4 arrondissements ayant le moins de lieux culturels sont :
CODE_POSTAL
75002    21
75007    29
75001    31
75003    47
Name: count, dtype: int64


Aucun des 4 arrondissements ayant les meilleures note pour leur culture ne se trouvent dans la liste des 4 arrondissements ayant le plus de lieux culturels. Il semble donc venir de cette analyse que la qualité prime sur la quantité de lieux culturels pour la satisfaction des habitants.

Les 4 arrondissements ayant les pires notes ne sont absolument pas ceux ayant le moins d'institutions culturelles non plus. La conclusion semble alors se confirmer. Cela va même encore plus loin : le 3ème arrondissement, qui est de loin le préféré des Parisiens sur le critère de la culture, est le 4ème arrondissement ayant le moins de lieux culturels !
